In [20]:
!pip install -q -U git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/accelerate.git git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops sentencepiece


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [21]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [22]:
import transformers

print(f"Running on transformers version: {transformers.__version__}")

Running on transformers version: 4.32.0.dev0


In [23]:
import torch
device = torch.device("cuda")

device

device(type='cuda')

In [24]:
import argparse
import os
import pandas as pd
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
)

#import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

from smart_open import smart_open
from tqdm.contrib.concurrent import process_map
from scipy.special import expit
from sklearn import metrics
from sklearn.metrics import classification_report
from datasets import load_dataset

In [38]:
from smart_open import smart_open
import ast
train_df = pd.read_csv('train.csv') 
test_df = pd.read_csv('test.csv')  

In [7]:
train_df.shape,test_df.shape

((7932, 3), (1989, 3))

In [8]:
train_df['len-labels']  = train_df['labels'].apply(len)

In [9]:
import re

def clean_tweet(tweet):
    # Remove special characters, links, and usernames
    tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
    tweet = re.sub(r"[^\w\s]", "", tweet)

    # Convert to lowercase
    tweet = tweet.lower()

    # Remove common stopwords specific to tweets
    stopwords = ["rt", "retweet"]
    tweet = ' '.join(word for word in tweet.split() if word not in stopwords)

    # Remove extra whitespace
    tweet = re.sub(r"\s+", " ", tweet).strip()

    return tweet

In [10]:
train_df['clean_text'] = train_df['text'].apply(clean_tweet)

In [11]:
test_df['clean_text'] = test_df['text'].apply(clean_tweet)

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(train_df['labels'])
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [13]:
train = pd.concat([train_df[['clean_text']], pd.DataFrame(labels)], axis=1)
train.columns = ['clean_text'] + list(mlb.classes_)
train.head(4) 

,clean_text,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
0,astrazeneca is made with the kidney cells of a...,0,0,0,1,0,0,0,0,0,0,0,0
1,it begins please find safe alternatives to thi...,0,0,0,0,0,0,0,0,0,0,1,0
2,we do not need any crazy twisted politician te...,0,0,0,0,0,0,0,1,0,0,0,0
3,i respect natalies view even if it differs fro...,0,0,0,0,1,0,0,0,0,0,0,0


In [14]:
labels_test = mlb.fit_transform(test_df['labels'])
labels_test

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [15]:
test = pd.concat([test_df[['clean_text']], pd.DataFrame(labels_test)], axis=1)
test.columns = ['clean_text'] + list(mlb.classes_)
test.head(4) 

,clean_text,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
0,well i mean congratulations covid19 for being ...,0,0,0,0,0,0,0,0,0,0,1,0
1,for those of us that do not wish a vaccine so ...,0,0,0,0,1,0,0,0,0,0,0,0
2,she has been trying to speak out writing lette...,0,0,0,0,0,0,0,0,0,1,1,0
3,im confused with the vaccine were still requir...,0,0,1,0,1,0,0,0,0,0,0,0


In [16]:
train.to_csv('train_processed.csv',index=False)
test.to_csv('test_processed.csv',index=False)

In [ ]:
# --------------

In [81]:
# train = pd.read_csv('train_processed.csv')
# test = pd.read_csv('test_processed.csv')

In [82]:
train.head(2)

,clean_text,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary
0,astrazeneca is made with the kidney cells of a...,0,0,0,1,0,0,0,0,0,0,0,0
1,it begins please find safe alternatives to thi...,0,0,0,0,0,0,0,0,0,0,1,0


In [90]:
test.head(2)

,clean_text,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary,labels
0,well i mean congratulations covid19 for being ...,0,0,0,0,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
1,for those of us that do not wish a vaccine so ...,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"


In [83]:
label_cols = list(train.columns[1:]) 

In [84]:
label_cols

['conspiracy',
 'country',
 'ineffective',
 'ingredients',
 'mandatory',
 'none',
 'pharma',
 'political',
 'religious',
 'rushed',
 'side-effect',
 'unnecessary']

In [85]:
label2id = dict()
id2label = dict()
for idx, col in enumerate(label_cols):
    id2label[idx] = col
    label2id[col] = idx

In [20]:
# train_temp.to_csv('train.csv',index=False)
# test_temp.to_csv('test.csv',index=False)

In [21]:
# import pickle
# with smart_open('s3://a207918-ml-workspace-taxalertingfsc8k-use1/tax-type-classification/bert_clfs_data/lbl2id.pickle', 'rb') as f:
#     label2id= pickle.load(f)
    
    
# with smart_open('s3://a207918-ml-workspace-taxalertingfsc8k-use1/tax-type-classification/bert_clfs_data/id2lbl.pickle', 'rb') as f:
#     id2label= pickle.load(f)
    
    
    
# with smart_open('s3://a207918-ml-workspace-taxalertingfsc8k-use1/tax-type-classification/bert_clfs_data/label_cols.pickle', 'rb') as f:
#     label_cols= pickle.load(f)

In [86]:
train['labels'] = list(train[label_cols].values)
test['labels'] = list(test[label_cols].values)

In [ ]:
train.head()

In [87]:
train['text_len'] = train['clean_text'].apply(lambda x: len(x.split(" ")))   

In [88]:
train['text_len'].min()

2

In [89]:
train['text_len'].max()

61

In [17]:
train[train['text_len'] <=5]

,clean_text,conspiracy,country,ineffective,ingredients,mandatory,none,pharma,political,religious,rushed,side-effect,unnecessary,labels,text_len
14,ah the mandatory vaccine wank,0,0,0,0,1,0,0,0,0,0,0,0,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",5
992,anti vaccination,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",2
1233,there is no safe vaccine,0,0,0,0,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",5
1449,the vaccine doesnt stop transmission,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",5
1680,useless vaccine,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2
3694,we dont need no vaccination,0,0,0,0,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",5
4898,no poison for me thanks,0,0,0,0,0,0,0,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]",5
4918,the vaccine doesnt stop transmission,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",5
5201,more vaccine bullshit,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",3
7147,say no to,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",3


In [21]:
batch_size = 32
model_name_or_path = "roberta-large"
peft_type = PeftType.PREFIX_TUNING
device = device
num_epochs = 3

In [22]:
peft_config = PrefixTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=128) #256
lr = 1e-2
MAX_LENGTH = 100 #256

In [23]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [24]:
train_encodings = tokenizer(train["clean_text"].values.tolist(), max_length=MAX_LENGTH, padding="longest",truncation=True,return_tensors='pt').to(device)

test_encodings = tokenizer(test["clean_text"].values.tolist(), max_length=MAX_LENGTH, padding="longest",truncation=True,return_tensors='pt').to(device)

In [25]:
train_labels = train["labels"].values.tolist()
test_labels = test["labels"].values.tolist()

In [26]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # labels need to be converted to float for multilabel loss calculation
        item['labels'] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.labels)

In [27]:
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [28]:
# sanity check
tokenizer.decode(train_dataset[0]["input_ids"])

/tmp/ipykernel_20/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


'[CLS]astrazeneca is made with the kidney cells of a little girl aborted back in the 70s[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]'

In [29]:
from transformers import Trainer, TrainingArguments, AdamW, get_cosine_schedule_with_warmup, EarlyStoppingCallback

In [30]:
class TaxTrainer(Trainer):
    """Custom loss function defined under this class to accommodate multilabel classification.
    Taken from: https://huggingface.co/docs/transformers/main_classes/trainer
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

In [31]:
from scipy.special import expit
import numpy as np
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.array(expit(preds) > thresh, dtype=int)
    macro_f1 = metrics.f1_score(labels, preds, average='macro', zero_division=0)
    micro_f1 = metrics.f1_score(labels, preds, average='micro', zero_division=0)
    weighted_f1 = metrics.f1_score(labels, preds, average='weighted', zero_division=0)
    samples_f1 = metrics.f1_score(labels, preds, average='samples', zero_division=0)
    acc_thresh = accuracy_thresh(preds, labels, thresh)
    return {"macro_f1":macro_f1, "micro_f1":micro_f1, 
            "weighted_f1":weighted_f1, "samples_f1":samples_f1,
            'accuracy_thresh': acc_thresh}

def accuracy_thresh(y_pred, y_true, thresh, sigmoid=True): 
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid: 
        y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.bool()).float().mean().item()

def get_binary_preds(y_probs, thresh):
    if y_probs > thresh:
        return 1
    else:
        return 0
get_binary_preds_vec = np.vectorize(get_binary_preds)

def generate_performance_metrics(test_dataset,label_classes):
    predictions = trainer.predict(test_dataset)
    all_preds = predictions.predictions
    all_preds = torch.from_numpy(all_preds).float()
    preds = all_preds.sigmoid()

#     thresh = args.thresh

    test_bin_preds = get_binary_preds_vec(preds.numpy(), thresh)

    tdf_1hot = pd.DataFrame(predictions.label_ids, columns=label_classes)
    labels_subset_ids = [label2id[c] for c in label_classes if tdf_1hot[c].sum()>0]
    labels_subset = [c for c in label_classes if tdf_1hot[c].sum()>0]

    all_metrics = classification_report(predictions.label_ids, test_bin_preds, digits=3, labels = labels_subset_ids,
                                       target_names=labels_subset, output_dict=True, zero_division=0)
    metrics_df = pd.DataFrame(all_metrics).transpose().reset_index()
    return metrics_df

In [32]:
num_labels = len(label_cols)
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, num_labels=num_labels, label2id=label2id, id2label=id2label , problem_type="multi_label_classification").to(device)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'classifier.weight', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 6,316,056 || all params: 412,528,664 || trainable%: 1.5310586999598166


In [33]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataset) * num_epochs),
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [34]:
batch_size = 8
# configure logging so we see training loss
logging_steps = len(train_dataset) // batch_size
args = TrainingArguments(
    output_dir="roberta",
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    learning_rate=lr,
    #fp16=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    #weight_decay=0.01,
    warmup_steps=50,
    logging_steps=logging_steps,
    gradient_accumulation_steps= 32,
    lr_scheduler_type='cosine',
    metric_for_best_model='macro_f1', #micro_f1
    load_best_model_at_end = True
)

In [35]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.001)] 
)

In [36]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
thresh=0.5
trainer.train() #341 best 

/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Macro F1,Micro F1,Weighted F1,Samples F1,Accuracy Thresh
1,No log,0.278367,0.053906,0.364159,0.202246,0.320597,0.898022
2,No log,0.182154,0.410894,0.609444,0.583617,0.565544,0.929655
3,No log,0.158430,0.576121,0.677981,0.654990,0.639936,0.940380
4,No log,0.150961,0.627146,0.694241,0.684196,0.660717,0.943942
5,No log,0.149855,0.634064,0.705519,0.699952,0.676722,0.944109
6,No log,0.150176,0.650664,0.705722,0.697134,0.670588,0.945701
7,No log,0.154579,0.625150,0.708014,0.699215,0.693481,0.942601
8,No log,0.151600,0.650057,0.713253,0.704782,0.688872,0.945157
9,No log,0.155636,0.652935,0.715113,0.712378,0.701425,0.944319
10,No log,0.153874,0.665540,0.723422,0.716741,0.704257,0.945827


/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tens

TrainOutput(global_step=465, training_loss=0.1393396685200353, metrics={'train_runtime': 5378.0061, 'train_samples_per_second': 22.123, 'train_steps_per_second': 0.086, 'total_flos': 2.1733021770912e+16, 'train_loss': 0.1393396685200353, 'epoch': 15.0})

In [79]:
thresh=0.3
trainer.train() 

/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Macro F1,Micro F1,Weighted F1,Samples F1,Accuracy Thresh
1,No log,0.260083,0.164401,0.456958,0.349031,0.425976,0.101978
2,No log,0.184315,0.512212,0.630159,0.616768,0.618552,0.101978
3,No log,0.162576,0.607907,0.682472,0.671384,0.675417,0.101978
4,No log,0.154669,0.651076,0.704713,0.702390,0.706385,0.101978
5,No log,0.150193,0.664175,0.708463,0.710582,0.711999,0.101978
6,No log,0.150967,0.652696,0.709703,0.703751,0.713139,0.101978
7,No log,0.148412,0.661918,0.713430,0.710441,0.716754,0.101978


/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tens

TrainOutput(global_step=217, training_loss=0.18258348807761196, metrics={'train_runtime': 2582.2818, 'train_samples_per_second': 21.502, 'train_steps_per_second': 0.084, 'total_flos': 1.01420768264256e+16, 'train_loss': 0.18258348807761196, 'epoch': 7.0})

In [80]:
test_result = trainer.evaluate()

/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [81]:
test_result #0.3

{'eval_loss': 0.15019282698631287,
 'eval_macro_f1': 0.6641747265247872,
 'eval_micro_f1': 0.7084627329192545,
 'eval_weighted_f1': 0.7105815589343221,
 'eval_samples_f1': 0.7119993296463883,
 'eval_accuracy_thresh': 0.10197754204273224,
 'eval_runtime': 25.3038,
 'eval_samples_per_second': 78.605,
 'eval_steps_per_second': 9.84,
 'epoch': 7.0}

In [65]:
test_result

{'eval_loss': 0.1515214741230011,
 'eval_macro_f1': 0.6765697742408637,
 'eval_micro_f1': 0.7306382978723405,
 'eval_weighted_f1': 0.7240499312227765,
 'eval_samples_f1': 0.716155522037875,
 'eval_accuracy_thresh': 0.9469582438468933,
 'eval_runtime': 25.5968,
 'eval_samples_per_second': 77.705,
 'eval_steps_per_second': 9.728,
 'epoch': 15.0}

In [82]:
test_performance_df = generate_performance_metrics(test_dataset,label_cols)

/tmp/ipykernel_22/739800847.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [68]:
test_performance_df.head(20) #new one

,index,precision,recall,f1-score,support
0,conspiracy,0.755102,0.381443,0.506849,97.0
1,country,0.638298,0.750000,0.689655,40.0
2,ineffective,0.754491,0.754491,0.754491,334.0
3,ingredients,0.757143,0.609195,0.675159,87.0
4,mandatory,0.746479,0.675159,0.709030,157.0
5,none,0.650485,0.531746,0.585153,126.0
6,pharma,0.695652,0.690196,0.692913,255.0
7,political,0.568421,0.432000,0.490909,125.0
8,religious,1.000000,0.769231,0.869565,13.0
9,rushed,0.754153,0.769492,0.761745,295.0


In [83]:
test_performance_df.head(20) #0.3

,index,precision,recall,f1-score,support
0,conspiracy,0.528846,0.567010,0.547264,97.0
1,country,0.571429,0.800000,0.666667,40.0
2,ineffective,0.681462,0.781437,0.728033,334.0
3,ingredients,0.682353,0.666667,0.674419,87.0
4,mandatory,0.735294,0.636943,0.682594,157.0
5,none,0.560000,0.555556,0.557769,126.0
6,pharma,0.576271,0.800000,0.669951,255.0
7,political,0.472393,0.616000,0.534722,125.0
8,religious,1.000000,0.692308,0.818182,13.0
9,rushed,0.667622,0.789831,0.723602,295.0


# Inference

In [25]:
from peft import PeftModel, PeftConfig

In [26]:
MODEL_PATH = 'roberta/checkpoint-341'
MAX_LENGTH = 512

In [27]:
config = PeftConfig.from_pretrained(MODEL_PATH)

In [28]:
config.base_model_name_or_path

'roberta-large'

In [29]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [ ]:
num_labels = len(label_cols)
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, num_labels=num_labels, label2id=label2id, id2label=id2label , problem_type="multi_label_classification").to(device)

In [39]:
# Load the Lora model
inference_model = PeftModel.from_pretrained(model, MODEL_PATH)

inference_model.to(device)
inference_model.eval()

PeftModelForSequenceClassification(
  (base_model): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 1024, padding_idx=1)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_features=1024, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaS

In [52]:
import numpy as np

In [91]:
final_test = pd.read_csv('test.csv')

In [43]:
import re

def clean_tweet(tweet):
    # Remove special characters, links, and usernames
    tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
    tweet = re.sub(r"[^\w\s]", "", tweet)

    # Convert to lowercase
    tweet = tweet.lower()

    # Remove common stopwords specific to tweets
    stopwords = ["rt", "retweet"]
    tweet = ' '.join(word for word in tweet.split() if word not in stopwords)

    # Remove extra whitespace
    tweet = re.sub(r"\s+", " ", tweet).strip()

    return tweet

In [92]:
final_test['clean_text'] = final_test['tweet'].apply(clean_tweet)

In [116]:
final_test.head()

,id,tweet,clean_text
0,1070378532260470789t,Study Links HPV Vaccine to Historically High I...,study links hpv vaccine to historically high i...
1,973746711964372993t,Deaths from tainted measles vaccine affecting ...,deaths from tainted measles vaccine affecting ...
2,1043031076787040257t,"Am apreciat un videoclip pe @YouTube, https://...",am apreciat un videoclip pe treatment and cure...
3,1066338147527741440t,VIDEO --&gt;&gt; MMR #Vaccine increase risk of...,video gtgt mmr vaccine increase risk of autism...
4,963522018544152576t,Oral Polio Vaccine: Infecting Unvaccinated Kid...,oral polio vaccine infecting unvaccinated kids...


In [117]:
item_list = [{
    'id': row['id'],
    'clean_text': row['clean_text']#,
    #'real_labels': list(row['labels'])
    } for idx, row in final_test.iterrows()]

In [118]:
thresh=0.5
batch_size = 8

In [119]:
def get_batched_predictions(item_list, batch_size, thresh):
    outputs_list = []
    for batch in tqdm([item_list[x:x + batch_size]
                      for x in range(0, len(item_list), batch_size)]):
        # tokenize
        encoded = tokenizer(
            [d['clean_text'] for d in batch],  # list of text inputs to encode
            max_length=512,  # Pad & truncate all sentences.
            padding=True,
            truncation=True,
            return_token_type_ids=False,
            return_tensors='pt'
            )

        # predict
        with torch.no_grad():
            outputs = inference_model(
                encoded['input_ids'].to(device),
                token_type_ids=None,
                attention_mask=encoded['attention_mask'].to(device)
            )
        probs = np.array(expit(outputs[0].to(device='cpu')))

        # append predictions
        for i, b in enumerate(batch):
            outputs_list.append({
                'id': b['id'],
                'probabilities': probs[i],
                'predictions': list(map(lambda x: int(x > thresh), probs[i]))#,
               # 'real_labels': b['real_labels']
                })

    return outputs_list

In [120]:
outputs_list = get_batched_predictions(item_list, batch_size, thresh)

100%|██████████| 61/61 [00:05<00:00, 10.88it/s]


In [125]:
predictions_list = dict()
for p in outputs_list:
    predictions_list[p['id']] = [{
        'indices_pred':[i for i, e in enumerate(p['predictions']) if e == 1]#,
        #'indices_real': [i for i, e in enumerate(p['real_labels']) if e == 1],
        }
    ] 

In [127]:
out_list = []
for i in tqdm(final_test['id']):
    df_items = {
    'id' : i,
    'pred_classes' : [id2label[k] for k in predictions_list[i][0]['indices_pred']]#,
   # 'actual_classes'  : [id2label[k] for k in predictions_list[i][0]['indices_real']]
    } 
    out_list.append(df_items)

100%|██████████| 486/486 [00:00<00:00, 253473.23it/s]


In [160]:
out_df = pd.DataFrame(out_list)

In [161]:
out_df.head()

,id,pred_classes
0,1070378532260470789t,[side-effect]
1,973746711964372993t,"[ingredients, side-effect]"
2,1043031076787040257t,[side-effect]
3,1066338147527741440t,[side-effect]
4,963522018544152576t,[]


In [162]:
out_df.explode(['pred_classes']).groupby(['pred_classes'])['id'].count()

pred_classes
conspiracy      26
country          5
ineffective     50
ingredients     58
mandatory       41
none            35
pharma          55
political       16
religious       11
rushed          40
side-effect    193
unnecessary     15
Name: id, dtype: int64

In [163]:
# Function to remove 'none' if it appears with another label
def remove_none(lst):
    if 'none' in lst and len(lst) > 1:
        lst.remove('none')
    return lst

# Apply the function to each list in 'column_name'
out_df['pred_classes'] = out_df['pred_classes'].apply(remove_none)

In [167]:
out_df.head()

,id,pred_classes
0,1070378532260470789t,[side-effect]
1,973746711964372993t,"[ingredients, side-effect]"
2,1043031076787040257t,[side-effect]
3,1066338147527741440t,[side-effect]
4,963522018544152576t,[]


In [168]:
out_df['pred_classes'] = out_df['pred_classes'].apply(' '.join)

In [169]:
out_df.head()

,id,pred_classes
0,1070378532260470789t,side-effect
1,973746711964372993t,ingredients side-effect
2,1043031076787040257t,side-effect
3,1066338147527741440t,side-effect
4,963522018544152576t,


In [173]:
out_df[out_df['pred_classes']==''].shape # predicting nothing

(38, 2)

In [176]:
out_df.to_csv('./roberta_prefix_tuning.csv',index=False)

In [7]:
import pandas as pd
import ast

In [9]:
df = pd.read_csv('roberta_prefix_tuning_predictions.csv')

In [15]:
df['pred_classes'] = df['pred_classes'].apply(lambda x: list(x))

TypeError: 'float' object is not iterable

In [68]:
df.shape

(486, 2)

In [16]:
df = df.rename(columns={'pred_classes': 'preds'})

In [70]:
df

,id,preds
0,1070378532260470789t,side-effect
1,973746711964372993t,ingredients side-effect
2,1043031076787040257t,side-effect
3,1066338147527741440t,side-effect
4,963522018544152576t,NaN
...,...,...
481,1099937642169405440t,side-effect
482,1536674508731518985t,side-effect
483,1183314494874968064t,side-effect
484,1327581896243556352t,side-effect


In [71]:
df.to_csv('./roberta_prefix_tuning_preds.csv',index=False)